In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from opencage.geocoder import OpenCageGeocode
import folium
from matplotlib.font_manager import FontProperties
from matplotlib.ticker import AutoMinorLocator
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import os

In [ ]:
egypt_df = pd.DataFrame(pd.read_csv("../assets/normalized_pyramid_data.csv"))

In [ ]:
def df_df(df):
    """
    Creates a dataframe containing only the dynasty's and total height of complexes for each dynasty.
        Input:
                df: The normalized data dataframe with all data about kings and queens.
        Output:
                A Matplotlib.pyplot triangle graph of total heigth per dynasty.
    """
    dynasty_heights_df=df[['dynasty','height']][pd.notnull(df['height'])]
    #handle pyramids who have ranges by choosing median
    dynasty_heights_df.at[18, 'height'] = 9.365
    dynasty_heights_df.at[50, 'height'] = 16.5
    #handles pyramid that has both a projected and actual height by choosing projected height
    dynasty_heights_df.at[22, 'height'] = 72

    #gets total heights for each dynasty
    dynasty_heights_df['height']=dynasty_heights_df['height'].astype(float)
    dynasty_heights_df=dynasty_heights_df.groupby('dynasty')['height'].sum().reset_index() 
    return dynasty_heights_df

def create_heights_plot(df):
    #generate plot
    fig, ax = plt.subplots(figsize=(15, 5))
    sns.set_style(style='ticks')
    plt.rcParams['font.family'] = 'Times New Roman'

    #plot data
    for i, (category, value) in enumerate(zip(df['dynasty'].to_list(), df['height'].to_list())):
        ax.fill([i-0.275, i, i+0.275], [0, value, 0], color='#d2ab84', alpha=1, edgecolor='white', linewidth=1.)
        #adds label with total height above each bar
        ax.text(i, value + 5, f'{value}', ha='center', va='bottom', fontsize=10, fontweight='bold')

    ax.set_xticks(range(len(df['dynasty'].to_list())))
    ax.set_xticklabels(df['dynasty'].to_list())
    plt.tick_params(axis='both', which='major', length=5, labelsize=10) 

    #add title and axis labels
    plt.title('Aggregated Heights Across Different Dynasties',fontweight='bold',pad=20, fontsize=12)
    plt.xlabel('Dynasty',fontweight='bold',labelpad=20, fontsize=10)
    plt.ylabel('Aggregated Heights (meters)', fontweight='bold',labelpad=20, fontsize=10)
    plt.show()


In [ ]:
def get_data(df):
    """
    Creates a dataframe containing only the name of sites, counts for kings and queens, country name, latitude, 
    and longitude. 

        Input:
                df: The normalized data dataframe with all data about kings and queens.
        Output:
                A dataframe with the name of rulers, year their reign started, and the length of their reign.
    """
    sites_rulers=df[['site_or_location','casing','superstructure_type','royal_status']][df['site_or_location']!='Unknown']
    sites_rulers = sites_rulers.groupby('site_or_location', sort=False).agg(
        kings_count=('royal_status', lambda x: sum(x == 'King')),
        queens_count=('royal_status', lambda x: sum(x == 'Queen'))
    ).reset_index()
    sites_rulers['country']=['Egypt']*9

    #ensure dashur only has one king listed
    sites_rulers.loc[3,'kings_count']=1

    #combine info about Saqqara into one
    sites_rulers=sites_rulers.drop([6,8]).reset_index(drop=True)
    sites_rulers.loc[0,'kings_count']=12
    sites_rulers.loc[0,'queens_count']=19

    #get latitude and longitude values
    key = '91fc143d65da4b8ba69d53e153d0fdef'
    geocoder = OpenCageGeocode(key)
    def geocode_address(row):
        location = geocoder.geocode(f"{row['site_or_location']},{row['country']}")
        if location and len(location):
            return location[0]['geometry']['lat'], location[0]['geometry']['lng']
        else:
            return None, None
        
    #add latitude and longitude values to df
    sites_rulers['latitude'], sites_rulers['longitude'] = zip(*sites_rulers.apply(geocode_address, axis=1))
    return sites_rulers

def get_zawiyet(site_df):
    """
    Creates a map for the Zawiyet site. After adding markers and labels to the map(only the marker 
    for Zawiyet has a label), a screenshot is taken of this map and saved.  

        Input:
                sites_df: The html for the map that only contains the sites 
        Output:
                A string of the html for the map
    """
    #create map
    zawiyet_map = folium.Map(location=[(site_df['latitude'].median())-1.95, (site_df['longitude'].median())-1.2], zoom_start=8.45)
    #set color of markers
    for index, row in site_df.iterrows():
            if 'North Saqqara' == row['site_or_location']:
                color = 'darkred'
            elif 'Saqqara' == row['site_or_location']:
                color = 'red'
            elif 'South Saqqara' == row['site_or_location']:
                color = 'lightred'
            elif row['site_or_location'] == 'Zawiyet el-Aryan':
                color = 'orange'
            elif row['site_or_location'] == 'Meidum':
                color = 'green'
            elif row['site_or_location'] == 'Dahshur':
                color = 'darkgreen'
            elif row['site_or_location'] == 'Giza':
                color = 'blue'
            elif row['site_or_location'] == 'Abu Roash':
                color = 'darkblue'
            else:
                color = 'purple'
            
            #add markers to map
            folium.Marker(
                    [row['latitude'], row['longitude']], 
                    icon=folium.Icon(color=color, icon='')
            ).add_to(zawiyet_map)

            #add label to marker for zawiyet
            if row['site_or_location'] == 'Zawiyet el-Aryan':
                folium.map.Marker(
                            [row['latitude'], row['longitude']],
                            icon=folium.DivIcon(
                            icon_size=(170,0),
                            icon_anchor=(170,0),
                            html=f"""
                                        <div style="font-size: 17px; color: {color};font-weight:bold;font-family:Times New Roman;background-color: white;text-align:center;border-radius: 5px; border: 1px solid black">
                                        <b> Site: {row['site_or_location']}<br>
                                        Kings: {row['kings_count']}; Queens: {row['queens_count']}
                                        </div>
                                    """
                            )
                        ).add_to(zawiyet_map)
    #save zawiyet map html
    zawiyet_map
    zawiyet_map.save('../images/zawiyet_map.html')

    #open zawiyet map html in chrome and take screenshot of it
    chrome_options = Options()
    service = Service('#chrome_driver_path')

    driver = webdriver.Chrome(service=service, options=chrome_options)

    file_path = os.path.abspath("#file_path_to_zawiyet_html")
    file_url = f"file://{file_path}"
    driver.get(file_url)

    driver.save_screenshot("../images/zawiyet_map.png")
    driver.quit()

def get_other_sites(df):
    """
    Creates a map with the other sites excluding Zawiyet. After markers are added to the map
    and labels are added to those markers, a string of the html of the map is created. 

        Input:
                sites_df: The html for the map that only contains the sites 
        Output:
                A string of the html for the map
    """

    #remove zawiyet from sites df
    sites_df=df.drop([1]).reset_index(drop=True)

    #create map with other sites
    egypt_map = folium.Map(location=[sites_df['latitude'].median()-0.15, sites_df['longitude'].median()-0.1], zoom_start=10)

    #set color of markers
    for index, row in sites_df.iterrows():
            if 'North Saqqara' == row['site_or_location']:
                color = 'darkred'
            elif 'Saqqara' == row['site_or_location']:
                color = 'red'
            elif 'South Saqqara' == row['site_or_location']:
                color = 'lightred'
            elif row['site_or_location'] == 'Meidum':
                color = 'green'
            elif row['site_or_location'] == 'Dahshur':
                color = 'darkgreen'
            elif row['site_or_location'] == 'Giza':
                color = 'blue'
            elif row['site_or_location'] == 'Abu Roash':
                color = 'darkblue'
            else:
                color = 'purple'

        #add markers to maps
            folium.Marker(
                    [row['latitude'], row['longitude']], 
                    icon=folium.Icon(color=color, icon='')
            ).add_to(egypt_map)

        #add labels to markers
            folium.map.Marker(
                    [row['latitude'], row['longitude']],
                    icon=folium.DivIcon(
                    icon_size=(170,0),
                    icon_anchor=(170,0),
                    html=f"""
                            <div style="font-size: 12.5px; color: {color};font-weight:bold;font-family:Times New Roman;background-color: white;text-align:center;border-radius: 5px; border: 1px solid black">
                            <b> Site: {row['site_or_location']}<br>
                            Kings: {row['kings_count']}; Queens: {row['queens_count']}
                            </div>
                            """
                        )
                    ).add_to(egypt_map)
    egypt_map_html=egypt_map.get_root().render()
    return egypt_map_html

def final_egypt_map(sites_html):
    """
    Creates a table and adds it to the current map that has markers and labels for each site. After creating 
    the html for the table with site name, kings name, and queens name, the table is added on top of the maap 
    the right. A screenshot is then taken of the updated map with the table and saved. 

        Input:
                sites_html: The html for the map that only contains the sites 
        Output:
                An image that is the screenshot of map of egypt with table on the right. 
    """

    #create table with kings and queens info for each site
    egypt_table_html = """
    <div id="table-container" style="position: absolute; top: 3px;bottom:0.1px; right: 15px; width: 250px; background-color: none; border: 1px solid #ccc; box-shadow: 2px 2px 10px rgba(0,0,0,0.5); z-index: 9999;text-align:center">
        <table style="width: 100%; border-collapse: collapse;">
            <thead>
                <tr>
                    <th style="border: 1px solid black; padding: 3px; background-color:#d5bdaf; font-family:Times New Roman; color: #000;">Site</th>
                    <th style="border: 1px solid black; padding: 3px; background-color:#d5bdaf; font-family:Times New Roman; color: #000;">Kings</th>
                    <th style="border: 1px solid black; padding: 3px; background-color:#d5bdaf; font-family:Times New Roman; color: #000;">Queens</th>
                </tr>
            </thead>
            <tbody>
                <tr>
                    <td style="border: 1px solid black; padding: 3px; background-color:#e3d5ca; font-family:Times New Roman; color:#002060;">Abu Roash​</td>
                    <td style="border: 1px solid black; padding: 3px; background-color:#e3d5ca; font-family:Times New Roman; color:#002060;">Djedefre ​</td>
                    <td style="border: 1px solid black; padding: 3px; background-color:#e3d5ca; font-family:Times New Roman; color:#002060;">​</td>
                </tr>
                <tr>
                    <td style="border: 1px solid black; padding: 3px; background-color:#f5ebe0; font-family:Times New Roman; color:#a02b93;">Abusir</td>
                    <td style="border: 1px solid black; padding: 3px; background-color:#f5ebe0; font-family:Times New Roman; color:#a02b93;">Sahure, Neferirkare,​Neferefre, Niuserre</td>
                    <td style="border: 1px solid black; padding: 3px; background-color:#f5ebe0; font-family:Times New Roman; color:#a02b93;">Khentkaus II, Reputnebu ​</td>
                </tr>
                <tr>
                    <td style="border: 1px solid black; padding: 3px;background-color:#e3d5ca; font-family:Times New Roman; color:#678244;">Dahshur </td>
                    <td style="border: 1px solid black; padding: 3px;background-color:#e3d5ca; font-family:Times New Roman; color:#678244;">Sneferu</td>
                    <td style="border: 1px solid black; padding: 3px;background-color:#e3d5ca; font-family:Times New Roman; color:#678244;"></td>
                </tr>
                <tr>
                    <td style="border: 1px solid black; padding: 3px; background-color:#f5ebe0; font-family:Times New Roman; color:#47b3ff;">Giza</td>
                    <td style="border: 1px solid black; padding: 3px; background-color:#f5ebe0;  font-family:Times New Roman; color:#47b3ff;">Khufu, Khafre,​Menkaure</td>
                    <td style="border: 1px solid black; padding: 3px; background-color:#f5ebe0;font-family:Times New Roman; color:#47b3ff;">Meretites I, Henutsen, Khamerernebty II, Khentkaus I</td>
                </tr>
                <tr>
                    <td style="border: 1px solid black; padding: 3px; background-color:#e3d5ca; font-family:Times New Roman; color:#76b82a;">Meidum</td>
                    <td style="border: 1px solid black; padding: 3px; background-color:#e3d5ca; font-family:Times New Roman; color:#76b82a;">Sneferu</td>
                    <td style="border: 1px solid black; padding: 3px; background-color:#e3d5ca; font-family:Times New Roman; color:#76b82a;"></td>
                </tr>
                <tr>
                    <td style="border: 1px solid black; padding: 3px; background-color:#f5ebe0; font-family:Times New Roman; color:#c00000;">North Saqqara</td>
                    <td style="border: 1px solid black; padding: 3px; background-color:#f5ebe0; font-family:Times New Roman; color:#c00000;">Pepy I, Djedkare</td>
                    <td style="border: 1px solid black; padding: 3px; background-color:#f5ebe0; font-family:Times New Roman; color:#c00000;">Inenek Inti, Nebuwenet, Ankhnespepy I, Ankhnespepy II, Meretites II, Mehaa, Behenu, Sebutet, Nedjeftet, Ankhnespepy III, Setibhor</td>
                </tr>
                <tr>
                    <td style="border: 1px solid black; padding: 3px; background-color:#e3d5ca; font-family:Times New Roman; color:#ff0000;">Saqqara</td>
                    <td style="border: 1px solid black; padding: 3px; background-color:#e3d5ca; font-family:Times New Roman; color:#ff0000;">Djoser, Sekhemkhet, Userkaf, Menkauhor,​Unas, Teti ​</td>
                    <td style="border: 1px solid black; padding: 3px; background-color:#e3d5ca; font-family:Times New Roman; color:#ff0000;">Neferhetepes, Sesheshet I, Khuit II, Iput</td>
                </tr>
                <tr>
                    <td style="border: 1px solid black; padding: 3px; background-color:#f5ebe0; font-family:Times New Roman; color:#f07a7a;">South Saqqara</td>
                    <td style="border: 1px solid black; padding: 3px; background-color:#f5ebe0; font-family:Times New Roman; color:#f07a7a;">Shepseskaf, Merenre, Pepy II, Ibi ​</td>
                    <td style="border: 1px solid black; padding: 3px; background-color:#f5ebe0; font-family:Times New Roman; color:#f07a7a;">Neith, Iput II, Wedjebten​</td>
                </tr>
                <tr>
                    <td style="border: 1px solid black; padding: 3px; background-color:#e3d5ca; font-family:Times New Roman; color:#e69a20;">Zawiyet el-Aryan​</td>
                    <td style="border: 1px solid black; padding: 3px; background-color:#e3d5ca; font-family:Times New Roman; color:#e69a20;">Nebka, Khaba ​</td>
                    <td style="border: 1px solid black; padding: 3px; background-color:#e3d5ca; font-family:Times New Roman; color:#e69a20;">​</td>
                </tr>
            </tbody>
        </table>
    </div>
    """

    #combine map and table
    map_table_html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="utf-8">
        <title>Egypt Map</title>
    </head>
    <body style="margin: 0; padding: 0; height: 100vh; position: relative;">
        <div id="map" style="height: 100%; width: 100%;">
            {sites_html}
        </div>
        {egypt_table_html}
    </body>
    </html>
    """
    #save html file that has both map and table
    with open("../images/egypt_map_with_table.html", "w") as file:
        file.write(map_table_html)

    #open egypt map html in chrome and take screenshot of it
    chrome_options = Options()
    service = Service('/#chome_driver_path')

    driver = webdriver.Chrome(service=service, options=chrome_options)

    file_path = os.path.abspath("#file_path_to_egypt_map_with_table")
    file_url = f"file://{file_path}"
    driver.get(file_url)

    time.sleep(5)
    driver.save_screenshot("../images/egypt_map_with_table.png")
    driver.quit()

egypt_sites_df=get_data(egypt_df)
# get_zawiyet(egypt_sites_df)
html=get_other_sites(egypt_sites_df)
final_egypt_map(html)



In [ ]:
def rulers_reign_data(df):       
        """
        Creates a dataframe containing only the name of rulers, year their reign started, and the length of their reign. 

        Input:
                df: The normalized data dataframe with all data about kings and queens
        Output:
                A dataframe with the name of rulers, year their reign started, and the length of their reign
        """
        #data
        ruler_reigns=df[['pyramid_owner','start_of_reign','length_of_reign']][(pd.notnull(df['start_of_reign']))&(df['pyramid_owner']!='Nebka?')].reset_index(drop=True)
        ruler_reigns=ruler_reigns.drop([3,4]).reset_index(drop=True)

        #Sneferu combined into one data point, Khentkaus I, and Iby added
        ruler_reigns.loc[len(ruler_reigns)]={'pyramid_owner':'Sneferu','start_of_reign':2599,'length_of_reign':48}
        ruler_reigns.loc[len(ruler_reigns)]={'pyramid_owner':'Khentkaus I','start_of_reign':2467,'length_of_reign':0}
        ruler_reigns.loc[len(ruler_reigns)]= {'pyramid_owner': 'Iby', 'start_of_reign': 2143, 'length_of_reign':2.083}
        ruler_reigns=ruler_reigns.reindex(index=[0,1,2,19,3,4,5,6,7,20,8,9,10,11,12,13,14,15,16,17,18,21]).reset_index(drop=True)

        #y-values for where names of kings should be added in 
        ruler_reigns['y']=[-2,-3,-4,-5,-2.5,-2,-3,-2.5,-3,-4,-5,-2,-3,-2,-3,-2,-2,-3,-4.5,-2,-3,-2]
        ruler_reigns[['start_of_reign']]*=-1
        ruler_reigns["start_of_reign"] = ruler_reigns["start_of_reign"].astype(int)
        return ruler_reigns

def create_timeline(df):
        """
        Creates a timeline with rulers listed on chronologically order based on start of their reign. On top of the timeline, 
        bars are added to represent the length of their reigns. Below some rulers in the timeline, more pyramids to indicate 
        number of queens pyramids they built. 
        
        Input:
                df: The dataframe with name of rulers, start of reign, and length of reign
        Output:
                A Matplotlib.pyplot timeline plot of the reign of rulers.
        """
        #plot
        plt.rcParams['font.family']='Times New Roman'
        fig, ax = plt.subplots(figsize=(18, 8))
        ax.set_facecolor('#E6EEFF')

        ax.spines[["left", "top", "right", "bottom"]].set_visible(False)
        ax.spines[["bottom"]].set_position(("axes", 0.5))
        ax.yaxis.set_visible(False)
        ax.set_title("Length of Reign of Rulers in Years (From Start of Reign)", pad=10, loc="center", fontsize=15)


        #timeline
        ax.plot(df['start_of_reign'], [0] * len(df['start_of_reign']), "-",
                color="black") 
        plt.tick_params(axis='x', which='major', width=0)
        xticks = list(range(min(df.start_of_reign), max(df.start_of_reign), 100))
        ax.set_xticks(xticks)
        ax.set_xticklabels([f"{abs(xtick)} BCE" for xtick in xticks],fontsize=10,fontweight='bold')
        ax.set_ylim(-10, 10)

        #timeline markers
        colors = []
        for owners in df['pyramid_owner']:
                if owners=='Khentkaus I':
                        colors.append("#ffb978")  
                else:
                        colors.append("#83b0e1")  
        for i, reign in enumerate(df['start_of_reign']):
                ax.plot([reign], [0], "^", markerfacecolor=colors[i], markersize=10, markeredgewidth=0.3,
                markeredgecolor='black')
                start_of_reign, owner, y = df["start_of_reign"][i], df["pyramid_owner"][i], df["y"][i]
                ax.annotate(owner, xy=(start_of_reign, -0.145), xytext=(start_of_reign, y),
                        arrowprops=dict(arrowstyle="-",color="#3f220f", linewidth=0.1),
                        ha="center",
                        fontsize=10
                )
                        #adds bars for length of reign on top of pyramids (Khentkaus I excluded- length of reign unknown)
                if owner!= 'Khentkaus I':
                        reign_length = df['length_of_reign'][i]
                        ymin = 0.5
                        ax.vlines(x=reign, ymin=ymin, ymax=((reign_length/11)+ymin), color='#d2ab84', linewidth=4)
                        ax.text(reign, ((reign_length/11)+ymin) + 0.02, f'{reign_length}', ha='center', va='bottom', fontsize=9, color='black')

        #add sneferu two pyramids
        sneferu_index=df['pyramid_owner'].loc[lambda x: x=='Sneferu'].index.to_list()
        sneferu_reign = df['start_of_reign'][sneferu_index[0]]
        ax.plot(sneferu_reign, -0.5, "^", markerfacecolor=colors[sneferu_index[0]],markersize=10, markeredgewidth=0.3,
                markeredgecolor='black')
        ax.plot(sneferu_reign, -1, "^", markerfacecolor=colors[sneferu_index[0]],markersize=10, markeredgewidth=0.3,
                markeredgecolor='black')

        #add Khufu 3 queen pyramids
        khufu_index=df['pyramid_owner'].loc[lambda x: x=='Khufu'].index.to_list()
        khufu_reign = df['start_of_reign'][khufu_index[0]]
        ax.plot(khufu_reign, -0.5, "^", markerfacecolor="#ffb978",markersize=10, markeredgewidth=0.3,
                markeredgecolor='black')
        ax.plot(khufu_reign, -1, "^", markerfacecolor="#ffb978",markersize=10, markeredgewidth=0.3,
                markeredgecolor='black')
        ax.plot(khufu_reign, -1.5, "^", markerfacecolor="#ffb978",markersize=10, markeredgewidth=0.3,
                markeredgecolor='black')

        #add Menkaure 3 queen pyramids
        menkaure_index=df['pyramid_owner'].loc[lambda x: x=='Menkaure'].index.to_list()
        menkaure_reign = df['start_of_reign'][menkaure_index[0]]
        ax.plot(menkaure_reign, -0.5, "^", markerfacecolor="#ffb978",markersize=10, markeredgewidth=0.3,
                markeredgecolor='black')
        ax.plot(menkaure_reign, -1, "^", markerfacecolor="#ffb978",markersize=10, markeredgewidth=0.3,
                markeredgecolor='black')
        ax.plot(menkaure_reign, -1.5, "^", markerfacecolor="#ffb978",markersize=10, markeredgewidth=0.3,
                markeredgecolor='black')

        #add Userkaf one queen pyramid
        userkaf_index=df['pyramid_owner'].loc[lambda x: x=='Userkaf'].index.to_list()
        userkaf_reign = df['start_of_reign'][userkaf_index[0]]
        ax.plot(userkaf_reign, -0.5, "^", markerfacecolor="#ffb978",markersize=10, markeredgewidth=0.3,
                markeredgecolor='black')

        #add Niuserre one queen pyramid
        niuserre_index=df['pyramid_owner'].loc[lambda x: x=='Niuserre'].index.to_list()
        niuserre_reign = df['start_of_reign'][niuserre_index[0]]
        ax.plot(niuserre_reign, -0.5, "^", markerfacecolor="#ffb978",markersize=10, markeredgewidth=0.3,
                markeredgecolor='black')

        #add Teti three queen pyramids
        teti_index=df['pyramid_owner'].loc[lambda x: x=='Teti'].index.to_list()
        teti_reign = df['start_of_reign'][teti_index[0]]
        ax.plot(teti_reign, -1, "^", markerfacecolor="#ffb978",markersize=10, markeredgewidth=0.3,
                markeredgecolor='black')
        ax.plot(teti_reign, -1.5, "^", markerfacecolor="#ffb978",markersize=10, markeredgewidth=0.3,
                markeredgecolor='black')
        ax.plot(teti_reign, -2, "^", markerfacecolor="#ffb978",markersize=10, markeredgewidth=0.3,
                markeredgecolor='black')

        #add Pepy I seven queen pyramids
        pepy_i_index=df['pyramid_owner'].loc[lambda x: x=='Pepy I'].index.to_list()
        pepy_i_reign = df['start_of_reign'][pepy_i_index[0]]
        ax.plot(pepy_i_reign, -0.5, "^", markerfacecolor="#ffb978",markersize=10, markeredgewidth=0.3,
                markeredgecolor='black')
        ax.plot(pepy_i_reign, -1, "^", markerfacecolor="#ffb978",markersize=10, markeredgewidth=0.3,
                markeredgecolor='black')
        ax.plot(pepy_i_reign, -1.5, "^", markerfacecolor="#ffb978",markersize=10, markeredgewidth=0.3,
                markeredgecolor='black')
        ax.plot(pepy_i_reign, -2, "^", markerfacecolor="#ffb978",markersize=10, markeredgewidth=0.3,
                markeredgecolor='black')
        ax.plot(pepy_i_reign, -2.5, "^", markerfacecolor="#ffb978",markersize=10, markeredgewidth=0.3,
                markeredgecolor='black')
        ax.plot(pepy_i_reign, -3, "^", markerfacecolor="#ffb978",markersize=10, markeredgewidth=0.3,
                markeredgecolor='black')
        ax.plot(pepy_i_reign, -3.5, "^", markerfacecolor="#ffb978",markersize=10, markeredgewidth=0.3,
                markeredgecolor='black')

        #add Pepy II three queen pyramids
        pepy_ii_index=df['pyramid_owner'].loc[lambda x: x=='Pepy II'].index.to_list()
        pepy_ii_reign =df['start_of_reign'][pepy_ii_index[0]]
        ax.plot(pepy_ii_reign, -0.5, "^", markerfacecolor="#ffb978",markersize=10, markeredgewidth=0.3,
                markeredgecolor='black')
        ax.plot(pepy_ii_reign, -1, "^", markerfacecolor="#ffb978",markersize=10, markeredgewidth=0.3,
                markeredgecolor='black')
        ax.plot(pepy_ii_reign, -1.5, "^", markerfacecolor="#ffb978",markersize=10, markeredgewidth=0.3,
                markeredgecolor='black')
        plt.show()
        
rulers_reign_df=rulers_reign_data(egypt_df)
create_timeline(rulers_reign_df)
